In [17]:
# Protocol
# sepcifies method which an object must provide to fulfill a role

In [18]:
#Protocol:[__getitem__]
class Vowels:
    def __getitem__(self, i):
        return 'AEIOU'[i]
    

In [19]:
v = Vowels()
v[0]

'A'

In [20]:
# Dynamic protocol
# defined by conventionm and describedin the documentation
# Static protocol
# explicit definition: a typing.Protocol subclass 



In [21]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

## Sequence like data structure [__len__, __getitem__] must be implemented
class FrenchDeck:
    ranks = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                       for rank in self.ranks]

    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]




In [22]:
from random import shuffle 
deck = FrenchDeck() 
# __setitem__ must be implemented 
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

In [ ]:
## Monkey patching -> manually patch a function 
def set_card(deck, position, card):
    deck._cards[position] = card 

FrenchDeck.__setitem__ = set_card

In [23]:
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

In [24]:
deck[:10]

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades'),
 Card(rank='5', suit='spades'),
 Card(rank='6', suit='spades'),
 Card(rank='7', suit='spades'),
 Card(rank='8', suit='spades'),
 Card(rank='9', suit='spades'),
 Card(rank='10', suit='spades'),
 Card(rank='J', suit='spades')]

In [25]:
## Goose typing
class Struggle:
    def __len__(self): return 23

from collections import abc 




In [26]:
isinstance(Struggle, abc.Sized)

False

In [28]:
from collections import namedtuple, abc

Card = namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(abc.MutableSequence):
    ranks = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split() 

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, value):
        self._cards[position] = value 

    def __delitem__(self, position):
        del self._cards[position]

    def inesert(self, position, value):
        self._cards.insert(position, value)




In [ ]:
## does not check at import
## checks at run
deck = FrenchDeck2()

TypeError: Can't instantiate abstract class FrenchDeck2 with abstract method insert

In [31]:
import abc 

class Tombola(abc.ABC):

    @abc.abstractmethod
    def load(self, iterable):
        """Add items from an iterable"""

    @abc.abstractmethod
    def pick(self):
        """Remove item at random, returning it.
        """

    def loaded(self):
        """Return 'True' if there's at least 1 item, 'False' otherwise"""

        return bool(self.inspect())
    
    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = [] 
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break 
        self.load(items)
        return tuple(items)

In [36]:
class Fake(Tombola):
    def pick(self):
        return 14
    # def load(self):
    #     return 12

In [37]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract method load

In [38]:
## Stacking decorators


In [39]:
class MyABC(abc.ABC):
    @classmethod
    @abc.abstractmethod
    def an_abstract_classmethod(cls):
        pass 

In [40]:
import random 

class BingoCage(Tombola):
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
    
    def pick(self):
        try:
            return self._items.pop() 
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    def __call__(self):
        self.pick()

        

In [43]:
a = BingoCage([1,2,3])

In [44]:
class LottoBlower(Tombola):

    def __init__(self, iterable):
        self._balls = list(iterable)
    
    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LottoBlower')
        return self._balls.pop(position)
    
    def loaded(self):
        return bool(self._balls)
    
    def inspect(self):
        return tuple(self._balls)

In [46]:
b = LottoBlower([1,2,3,3,3,]
                )

In [47]:
@Tombola.register
class TomboList(list):
    def pick(self):
        if self:
            position = random.randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')
    
    load = list.extend

    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(self)
    

In [48]:
t = TomboList(range(100)) 
isinstance(t, Tombola)

True

In [50]:
from abc import ABCMeta
## This is how the abc checks if one class is instance of an abc class 
class Sized(metaclass = ABCMeta):

    __slots__ = () 

    @abc.abstractmethod
    def __len__(self):
        return 0
    
    @classmethod
    def __subclasshook__(cls, C):
        if cls is Sized:
            if any("__len__" in B.__dict__ for B in C.__mro__):
                return True
        return NotImplemented
    

In [51]:
def double(x):
    return x * 2 


In [52]:
double([1,2,3,4])

[1, 2, 3, 4, 1, 2, 3, 4]

In [54]:
from fractions import Fraction 
double(Fraction(2,5))

Fraction(4, 5)

In [56]:
from typing import TypeVar, Protocol 

T = TypeVar('T')

class Repeatable(Protocol):
    def __mul__(self:T, repeat_count: int) -> T:
        pass 

RT = TypeVar('RT', bound=Repeatable)

def double(x: RT) -> RT:
    return x * 2 

In [57]:
from typing import runtime_checkable

@runtime_checkable
class SupportComplex(Protocol):
    """"""
    __slots__ = () 

    @abc.abstractmethod
    def __complex__(self) -> complex:
        pass 

In [58]:
import numpy as np 

c64 = np.complex64(3+4j)
isinstance(c64, complex)



False

In [59]:
isinstance(c64, SupportComplex)

True

In [60]:
c = complex(c64)

In [61]:
import sys
sys.version

'3.11.10 (main, Oct  3 2024, 02:37:52) [Clang 14.0.6 ]'

In [62]:
c = 3 + 4j
 

In [64]:
c.__float__

AttributeError: 'complex' object has no attribute '__float__'

In [65]:
from typing import SupportsFloat

isinstance(c, SupportsFloat)

False